In [1]:
import numpy as np
import pandas as pd

In [2]:
# Reading the datasets
df_train = pd.read_csv('train_Tweet.csv')
df_test = pd.read_csv('test_Tweet.csv')

display(df_train)
display(df_test)


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,416863ce47,All alone in this old house again. Thanks for...,positive
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive


In [3]:
df_train.describe()


,textID,text,selected_text,sentiment
count,27481,27480,27480,27481
unique,27481,27480,22463,3
top,cb774db0d1,"I`d have responded, if I were going",good,neutral
freq,1,1,199,11118


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   textID         27481 non-null  object
 1   text           27480 non-null  object
 2   selected_text  27480 non-null  object
 3   sentiment      27481 non-null  object
dtypes: object(4)
memory usage: 858.9+ KB


In [5]:
df_train.isnull().sum()

textID           0
text             1
selected_text    1
sentiment        0
dtype: int64

In [6]:
df_test.isnull().sum()

textID       0
text         0
sentiment    0
dtype: int64

In [7]:
df_train.dropna(inplace=True)


In [8]:
# Print the first row in the training dataset
print("First Row in the Training Dataset:")
print(df_train.iloc[0])

print('-' * 30)

# Count the number of rows where 'text' is equal to 'selected_text'
matching_rows = len(df_train[df_train['text'] == df_train['selected_text']])
print(f"Number of Matching Rows: {matching_rows}")

First Row in the Training Dataset:
textID                                     cb774db0d1
text              I`d have responded, if I were going
selected_text     I`d have responded, if I were going
sentiment                                     neutral
Name: 0, dtype: object
------------------------------
Number of Matching Rows: 7283


In [9]:
# Check for empty strings - train set
blanks_train = []

for i, tid, t, tst, snt in df_train.itertuples():
    if type(t) == str and t.isspace():
        blanks_train.append(i)
        
if len(blanks_train) > 0:
    print(len(blanks_train))
    df_train.drop(blanks_train, inplace=True)

In [10]:
# Check for empty strings - test set
blanks_test = []

for i, tid, t, snt in df_test.itertuples():
    if type(t) == str and t.isspace():
        blanks_test.append(i)
        
if len(blanks_test) > 0:
    print(len(blanks_test))
    df_train.drop(blanks_test, inplace=True)


In [11]:
# Check for blanacement
df_train['sentiment'].value_counts()


neutral     11117
positive     8582
negative     7781
Name: sentiment, dtype: int64

In [12]:
# Performing T-Test
from scipy import stats

df_train['text_length'] = df_train['text'].apply(lambda t: len(str(t)))

positive_text_lengths = df_train[df_train['sentiment'] == 'positive']['text_length']
negative_text_lengths = df_train[df_train['sentiment'] == 'negative']['text_length']
neutral_text_lengths = df_train[df_train['sentiment'] == 'neutral']['text_length']

# Perform ANOVA test
f_statistic, p_value = stats.f_oneway(positive_text_lengths, negative_text_lengths, neutral_text_lengths)

# Print the results
print("ANOVA Test Results:")
print(f"F-statistic: {f_statistic}")
print(f"P-value: {p_value}")

# Interpret the results
alpha = 0.05  # Set your significance level
if p_value < alpha:
    print("The means of at least two groups are significantly different.")
else:
    print("There is no significant difference in the means of the groups.")

ANOVA Test Results:
F-statistic: 72.2127709711816
P-value: 5.254438748898152e-32
The means of at least two groups are significantly different.


In [13]:
# Defining X and y
X_train = df_train['text']
y_train = df_train['sentiment']

X_test = df_test['text']
y_test = df_test['sentiment']

In [14]:
# Handle y categorical data
y_train_categorical = pd.get_dummies(y_train, dtype=int)
y_test_categorical = pd.get_dummies(y_test, dtype=int)

In [15]:
y_train_categorical

,negative,neutral,positive
0,0,1,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
27476,1,0,0
27477,1,0,0
27478,0,0,1
27479,0,0,1


In [16]:
y_test_categorical

,negative,neutral,positive
0,0,1,0
1,0,0,1
2,1,0,0
3,0,0,1
4,0,0,1
...,...,...,...
3529,1,0,0
3530,0,0,1
3531,1,0,0
3532,0,0,1


In [17]:
# Import basic libraries
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

In [18]:
# SVM model
pipeline = Pipeline([('tfidf', TfidfVectorizer()), ('classifier', LinearSVC())])

grid_params_svm = {
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__stop_words': [None, 'english'],
    'classifier__C': [0.01, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 5.0, 10.0],
}

svm_model = GridSearchCV(pipeline, grid_params_svm, cv=5, scoring='accuracy')

svm_model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('classifier', LinearSVC())]),
             param_grid={'classifier__C': [0.01, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5,
                                           5.0, 10.0],
                         'tfidf__ngram_range': [(1, 1), (1, 2)],
                         'tfidf__stop_words': [None, 'english']},
             scoring='accuracy')

In [19]:
# NB model
pipeline = Pipeline([('tfidf', TfidfVectorizer()), ('classifier', MultinomialNB())])

grid_params_nb = {
    'tfidf__ngram_range': [(1,1), (1,2)],
    'tfidf__stop_words': [None, 'english'],
    'classifier__alpha': [0.01, 0.1, 0.5, 1.0, 1.5, 2.0, 2.5, 5.0, 10.0]
}

nb_model = GridSearchCV(pipeline, grid_params_nb, cv=5, scoring='accuracy')

nb_model.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('classifier', MultinomialNB())]),
             param_grid={'classifier__alpha': [0.01, 0.1, 0.5, 1.0, 1.5, 2.0,
                                               2.5, 5.0, 10.0],
                         'tfidf__ngram_range': [(1, 1), (1, 2)],
                         'tfidf__stop_words': [None, 'english']},
             scoring='accuracy')

In [20]:
from sklearn.metrics import classification_report, confusion_matrix


In [21]:
# Define a function to show_metrics
def show_metrics(y_true, y_pred, grid=None):
    print(classification_report(y_true, y_pred))
    print('-' * 30)
    print(confusion_matrix(y_true, y_pred))
    
    if grid:
        print('-' * 30)
        print(grid.best_params_)


In [22]:
# SVM model
best_svm = svm_model.best_estimator_
y_pred_svm = best_svm.predict(X_test)

show_metrics(y_test, y_pred_svm)

              precision    recall  f1-score   support

    negative       0.69      0.68      0.68      1001
     neutral       0.66      0.67      0.66      1430
    positive       0.75      0.76      0.76      1103

    accuracy                           0.70      3534
   macro avg       0.70      0.70      0.70      3534
weighted avg       0.70      0.70      0.70      3534

------------------------------
[[677 268  56]
 [254 951 225]
 [ 46 214 843]]


In [23]:
# SVM model
best_nb = nb_model.best_estimator_
y_pred_nb = best_nb.predict(X_test)

show_metrics(y_test, y_pred_nb)

              precision    recall  f1-score   support

    negative       0.74      0.44      0.55      1001
     neutral       0.54      0.80      0.64      1430
    positive       0.76      0.57      0.65      1103

    accuracy                           0.63      3534
   macro avg       0.68      0.60      0.62      3534
weighted avg       0.67      0.63      0.62      3534

------------------------------
[[ 436  533   32]
 [ 123 1145  162]
 [  28  447  628]]


In [24]:
def save_model(model, prefix=''):
    import joblib
    from datetime import datetime

    # Get the current date and time as a string to define the file name
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{prefix}model_{current_datetime}.joblib"

    joblib.dump(model, filename)

    print(f"Model saved to {filename}")

In [25]:
save_model(nb_model, prefix='nb_')
save_model(svm_model, prefix='svm_')

Model saved to nb_model_2024-02-08_22-19-41.joblib
Model saved to svm_model_2024-02-08_22-19-41.joblib
